In [22]:
import pandas as pd
import visual as vs
import numpy as np
import importlib
import matplotlib.pyplot as plt
import preprocess as datapre

importlib.reload(vs)
importlib.reload(datapre)

<module 'preprocess' from 'D:\\GitHub\\MachineLearning\\capstone\\solutions\\preprocess.py'>

## 数据探索
来自于Kaggle名为旧金山罪案类型分类的数据集，该数据集分为训练集和测试集，训练集包含878049个带标签样本，测试集包含884262个未带标签样本。  
运行下边代码加载训练集和测试集。

In [23]:
train_data = pd.read_csv('../datasets/train.csv', parse_dates=['Dates'])
test_data = pd.read_csv('../datasets/test.csv', parse_dates=['Dates'])

### 显示训练集部分样本

In [24]:
train_data.head(4)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873


### 显示测试集部分样本

In [25]:
test_data.head(1)

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051


### 删除训练集中无用字段
训练集中的'Descript'，'Resolution'两个属性无意义，直接删除

In [26]:
# 删除无用字段 'Descript' 'Resolution'
train_data = train_data.drop(columns=['Descript', 'Resolution'])

### 处理数据集中的缺失值

In [27]:
train_data.isnull().any(axis=0)

Dates         False
Category      False
DayOfWeek     False
PdDistrict    False
Address       False
X             False
Y             False
dtype: bool

In [28]:
train_data.isna().any(axis=0)

Dates         False
Category      False
DayOfWeek     False
PdDistrict    False
Address       False
X             False
Y             False
dtype: bool

**确定训练集中没有缺失数据。**

### 特征提取

#### Dates解析为年月日时

In [29]:
train_data = datapre.extra_dates(train_data)

#### DayOfWeek转化为数字

In [30]:
from sklearn.preprocessing import LabelEncoder
DayOfWeekEnc = LabelEncoder()
train_data['DayOfWeekID'] = DayOfWeekEnc.fit_transform(train_data['DayOfWeek'])

#### PdDistrict转化为数字

In [31]:
PdDistrictEnc = LabelEncoder()
train_data['PdDistrictID'] = PdDistrictEnc.fit_transform(train_data['PdDistrict'])

#### 从Address中提取是否含有Block字段作为特征

In [32]:
train_data = datapre.extra_address_for_block(train_data)

#### 从Address中提取地址后缀作为特征
- 标准的地址都会含有一个简写后缀表示道路的类型，直接解析后缀，如："200 Block of INTERSTATE80 HY"解析为"HY"。
- 对于路口则会表示成'XX ST / YY ST'，直接解析为"CrossRoad"，如："STCHARLES AV / 19TH AV"解析为"CrossRoad"。
- 对于直接含有道路类型全名的地址也要进行解析。如："0 Block of AVENUE OF THE PALMS"中的"AVENUE"就是道路类型。
- 对于上述三种方式都无法解析，则直接设置为"Unkown"。  

根据直觉判断，不同类型的道路发生各种类型犯罪的分布是不一样的。如：铁路附近发生自杀案件的概率普遍高于其他案件。

In [33]:
train_data['PositionType'] = None
suffixs = ["AL","AV","BL","CR","CT","DR","EX","HWY","HY","LN","PL","PZ","RD","ST","TR","WY","WAY"]
suffix_names = ["Alley","Avenue","Boulevard","Circle","Court","Drive","Expressway","Highway","Highway",
                "Lane","Place","Plaza","Road","Street","Terrace","Way","Way"]
cross_road = "CrossRoad" # 交叉路口，含有/的
unkown_road = "Unkown"

# 设置交叉路口
train_data.loc[train_data['Address'].str.contains('/'), 'PositionType'] = cross_road

# 查找缩写并设置
for a in suffixs:
    train_data.loc[(train_data['Address'].str.contains('/') == False) 
                   & (train_data['Address'].str.contains(' '+a+'$')), 'PositionType'] = a
    
# 查找全程并设置
for i,d in enumerate(suffix_names):
    train_data.loc[(train_data['PositionType'].isna())
                   & (train_data['Address'].str.contains(d, case=False)), 'PositionType'] = suffixs[i]
    
# 无法解析的均设置为Unkown
train_data.loc[(train_data['PositionType'].isna()), 'PositionType'] = unkown_road

In [34]:
train_data.columns

Index(['Dates', 'Category', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y',
       'Year', 'Month', 'Day', 'Hour', 'DayOfWeekID', 'PdDistrictID',
       'HasBlock', 'PositionType'],
      dtype='object')

#### PositionType转化为数字

In [35]:
PositionTypeEnc = LabelEncoder()
train_data['PositionType'] = PositionTypeEnc.fit_transform(train_data['PdDistrict'])

#### 删除含有异常值的样本
属性'Category'、'PdDistrict'为类别，存在既有意义。属性'Address'为自由字符串，存在既有意义。
- Dates，需要在特定区间内
- DayOfWeek，只能有七种类型
- X，Y，

In [ ]:
X_bound = [-122.52, -122.35]
Y_bound = [37.70, 37.85]

In [ ]:
train_data.shape

(878049, 15)

In [ ]:
train_data.loc[(train_data['X'] > X_bound[0]) & (train_data['X'] < X_bound[1]) 
               & (train_data['Y'] > Y_bound[0]) & (train_data['Y'] < Y_bound[1])].shape

(877982, 15)

#### 设置训练模型特征名字

In [ ]:
feature_names = ['Year', 'Month', 'Day', 'Hour', 'DayOfWeekID', 'PdDistrictID', 'HasBlock', 'PositionType']

### 使用KNN算法进行分类
使用KNN算法以X、Y经纬度作为特征对训练集进行分类，将分类结果作为新的特征并入到训练集中。

### 使用XGBoost作为预测算法

In [ ]:
# 取出训练集中的预测目标字段
target = train_data['Category']
features = train_data[feature_names]

#### 开始训练模型

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
LabelEncTarget = LabelEncoder()
target = LabelEncTarget.fit_transform(target)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
print('X_train has {} samples.'.format(X_train.shape[0]))
print('X_test has {} samples.'.format(X_test.shape[0]))

DTrain_X = xgb.DMatrix(data=X_train, label=y_train)
DTest_X = xgb.DMatrix(data=X_test, label=y_test)

X_train has 702439 samples.
X_test has 175610 samples.


In [ ]:
from itertools import product

def products(data):
    results = [[]]
    for a in data:
        results = [x+[y] for x in results for y in a]
    for a in results:
        yield tuple(a)
        
def extra_params(param, param_grid):
    param_names = list(param_grid)
    params = []
    filenames = []
    param_sets = products(param_grid.values())
    for a in param_sets:
        b = param.copy()
        for key,value in zip(param_names, a):
            b[key] = value
        params.append(b)
        filenames.append('_'.join([str(i) for i in a]))
    return params,filenames


In [ ]:
import time
import datetime
from os import makedirs, system

param = {}
param['objective'] = 'multi:softprob'
# scale weight of positive examples
# param['eta'] = 0.01
# param['max_depth'] = 6
# param['silent'] = 1
# param['nthread'] = 8
param['num_class'] = len(LabelEncTarget.classes_)

param['eval_metric'] = 'mlogloss'
param['tree_method'] = 'gpu_hist'

evallist = [(DTrain_X, 'train'), (DTest_X, 'Test')]

param_grid = {
    'eta' : [0.01, 0.1],
    'max_depth': [3,6,9,10,20],
    'subsample' : [0.5, 0.7, 1.0],
    'grow_policy' : ["depthwise", "lossguide"],
}

# 参数和模型保存文件名
params,files = extra_params(param, param_grid)

# 模型保存文件路径
model_path = "../models/"+str(datetime.datetime.utcnow()).replace(':','_').replace(' ', '_').replace('.', '_')
makedirs(model_path)

# 模型文件名的含义
modelfileformat = "-".join(list(param_grid))
system("echo 123 > " + model_path + "/" + modelfileformat)
 
start = time.time()
result_data = {'best_iteration' : [],
               'best_score' : []}
num_round = 5000
for i,a in enumerate(params):
    bst =xgb.train(a, DTrain_X, num_round, evallist, early_stopping_rounds=10)
    bst.save_model(model_path + "/" + files[i] + ".model")
    result_data['best_iteration'].append(bst.best_iteration)
    result_data['best_score'].append(bst.best_score)

results = pd.DataFrame(index=files, data=result_data)
results.to_csv(model_path + "/" + "result.csv")

print('GPU Training Time: %s seconds.' % (str(time.time() - start)))

[0]	train-mlogloss:3.64613	Test-mlogloss:3.64615
Multiple eval metrics have been passed: 'Test-mlogloss' will be used for early stopping.

Will train until Test-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:3.62935	Test-mlogloss:3.62938
[2]	train-mlogloss:3.61315	Test-mlogloss:3.6132
[3]	train-mlogloss:3.59748	Test-mlogloss:3.59754
[4]	train-mlogloss:3.58231	Test-mlogloss:3.58239
[5]	train-mlogloss:3.56762	Test-mlogloss:3.56771
[6]	train-mlogloss:3.55334	Test-mlogloss:3.55345
[7]	train-mlogloss:3.53935	Test-mlogloss:3.53946
[8]	train-mlogloss:3.5259	Test-mlogloss:3.52603
[9]	train-mlogloss:3.51282	Test-mlogloss:3.51296
[10]	train-mlogloss:3.50006	Test-mlogloss:3.5002
[11]	train-mlogloss:3.48765	Test-mlogloss:3.4878
[12]	train-mlogloss:3.47555	Test-mlogloss:3.47571
[13]	train-mlogloss:3.46376	Test-mlogloss:3.46393
[14]	train-mlogloss:3.45218	Test-mlogloss:3.45235
[15]	train-mlogloss:3.44084	Test-mlogloss:3.44102
[16]	train-mlogloss:3.42984	Test-mlogloss:3.43003
[17]	train-ml

[161]	train-mlogloss:2.78304	Test-mlogloss:2.7832
[162]	train-mlogloss:2.78118	Test-mlogloss:2.78133
[163]	train-mlogloss:2.77933	Test-mlogloss:2.77949
[164]	train-mlogloss:2.77751	Test-mlogloss:2.77767
[165]	train-mlogloss:2.77569	Test-mlogloss:2.77584
[166]	train-mlogloss:2.77388	Test-mlogloss:2.77404
[167]	train-mlogloss:2.7721	Test-mlogloss:2.77225
[168]	train-mlogloss:2.77031	Test-mlogloss:2.77046
[169]	train-mlogloss:2.76855	Test-mlogloss:2.7687
[170]	train-mlogloss:2.76681	Test-mlogloss:2.76696
[171]	train-mlogloss:2.76506	Test-mlogloss:2.76521
[172]	train-mlogloss:2.76335	Test-mlogloss:2.76351
[173]	train-mlogloss:2.76166	Test-mlogloss:2.76181
[174]	train-mlogloss:2.75997	Test-mlogloss:2.76012
[175]	train-mlogloss:2.7583	Test-mlogloss:2.75845
[176]	train-mlogloss:2.75664	Test-mlogloss:2.75679
[177]	train-mlogloss:2.75497	Test-mlogloss:2.75512
[178]	train-mlogloss:2.75334	Test-mlogloss:2.75348
[179]	train-mlogloss:2.75173	Test-mlogloss:2.75187
[180]	train-mlogloss:2.75012	Test-m

[323]	train-mlogloss:2.60902	Test-mlogloss:2.60922
[324]	train-mlogloss:2.60845	Test-mlogloss:2.60865
[325]	train-mlogloss:2.60789	Test-mlogloss:2.60808
[326]	train-mlogloss:2.60732	Test-mlogloss:2.60751
[327]	train-mlogloss:2.60677	Test-mlogloss:2.60696
[328]	train-mlogloss:2.60621	Test-mlogloss:2.60641
[329]	train-mlogloss:2.60566	Test-mlogloss:2.60585
[330]	train-mlogloss:2.60511	Test-mlogloss:2.60531
[331]	train-mlogloss:2.60456	Test-mlogloss:2.60476
[332]	train-mlogloss:2.60401	Test-mlogloss:2.60421
[333]	train-mlogloss:2.60347	Test-mlogloss:2.60367
[334]	train-mlogloss:2.60294	Test-mlogloss:2.60314
[335]	train-mlogloss:2.60241	Test-mlogloss:2.60261
[336]	train-mlogloss:2.60188	Test-mlogloss:2.60207
[337]	train-mlogloss:2.60135	Test-mlogloss:2.60154
[338]	train-mlogloss:2.60083	Test-mlogloss:2.60103
[339]	train-mlogloss:2.60031	Test-mlogloss:2.60051
[340]	train-mlogloss:2.5998	Test-mlogloss:2.6
[341]	train-mlogloss:2.59929	Test-mlogloss:2.59948
[342]	train-mlogloss:2.59877	Test-ml

[485]	train-mlogloss:2.55009	Test-mlogloss:2.5504
[486]	train-mlogloss:2.54987	Test-mlogloss:2.55018
[487]	train-mlogloss:2.54965	Test-mlogloss:2.54996
[488]	train-mlogloss:2.54944	Test-mlogloss:2.54975
[489]	train-mlogloss:2.54923	Test-mlogloss:2.54954
[490]	train-mlogloss:2.54902	Test-mlogloss:2.54934
[491]	train-mlogloss:2.54881	Test-mlogloss:2.54912
[492]	train-mlogloss:2.54859	Test-mlogloss:2.54891
[493]	train-mlogloss:2.54838	Test-mlogloss:2.5487
[494]	train-mlogloss:2.54817	Test-mlogloss:2.5485
[495]	train-mlogloss:2.54797	Test-mlogloss:2.5483
[496]	train-mlogloss:2.54777	Test-mlogloss:2.5481
[497]	train-mlogloss:2.54757	Test-mlogloss:2.5479
[498]	train-mlogloss:2.54737	Test-mlogloss:2.5477
[499]	train-mlogloss:2.54717	Test-mlogloss:2.5475
[500]	train-mlogloss:2.54697	Test-mlogloss:2.5473
[501]	train-mlogloss:2.54677	Test-mlogloss:2.5471
[502]	train-mlogloss:2.54657	Test-mlogloss:2.54691
[503]	train-mlogloss:2.54637	Test-mlogloss:2.54671
[504]	train-mlogloss:2.54617	Test-mloglos

[647]	train-mlogloss:2.52641	Test-mlogloss:2.52707
[648]	train-mlogloss:2.52631	Test-mlogloss:2.52698
[649]	train-mlogloss:2.52622	Test-mlogloss:2.52688
[650]	train-mlogloss:2.52612	Test-mlogloss:2.52679
[651]	train-mlogloss:2.52603	Test-mlogloss:2.5267
[652]	train-mlogloss:2.52594	Test-mlogloss:2.52661
[653]	train-mlogloss:2.52584	Test-mlogloss:2.52652
[654]	train-mlogloss:2.52575	Test-mlogloss:2.52643
[655]	train-mlogloss:2.52565	Test-mlogloss:2.52634
[656]	train-mlogloss:2.52556	Test-mlogloss:2.52625
[657]	train-mlogloss:2.52547	Test-mlogloss:2.52616
[658]	train-mlogloss:2.52537	Test-mlogloss:2.52607
[659]	train-mlogloss:2.52528	Test-mlogloss:2.52598
[660]	train-mlogloss:2.52519	Test-mlogloss:2.52589
[661]	train-mlogloss:2.5251	Test-mlogloss:2.52581
[662]	train-mlogloss:2.52501	Test-mlogloss:2.52572
[663]	train-mlogloss:2.52492	Test-mlogloss:2.52563
[664]	train-mlogloss:2.52483	Test-mlogloss:2.52554
[665]	train-mlogloss:2.52475	Test-mlogloss:2.52546
[666]	train-mlogloss:2.52466	Test

...

[2348]	train-mlogloss:2.48867	Test-mlogloss:2.49607
[2349]	train-mlogloss:2.48866	Test-mlogloss:2.49606
[2350]	train-mlogloss:2.48866	Test-mlogloss:2.49606
[2351]	train-mlogloss:2.48865	Test-mlogloss:2.49606
[2352]	train-mlogloss:2.48864	Test-mlogloss:2.49606
[2353]	train-mlogloss:2.48863	Test-mlogloss:2.49605
[2354]	train-mlogloss:2.48862	Test-mlogloss:2.49605
[2355]	train-mlogloss:2.48862	Test-mlogloss:2.49604
[2356]	train-mlogloss:2.48861	Test-mlogloss:2.49604
[2357]	train-mlogloss:2.4886	Test-mlogloss:2.49604
[2358]	train-mlogloss:2.48859	Test-mlogloss:2.49603
[2359]	train-mlogloss:2.48859	Test-mlogloss:2.49603
[2360]	train-mlogloss:2.48858	Test-mlogloss:2.49603
[2361]	train-mlogloss:2.48857	Test-mlogloss:2.49602
[2362]	train-mlogloss:2.48856	Test-mlogloss:2.49602
[2363]	train-mlogloss:2.48856	Test-mlogloss:2.49601
[2364]	train-mlogloss:2.48855	Test-mlogloss:2.49601
[2365]	train-mlogloss:2.48854	Test-mlogloss:2.49601
[2366]	train-mlogloss:2.48853	Test-mlogloss:2.496
[2367]	train-ml

[2507]	train-mlogloss:2.48748	Test-mlogloss:2.4955
[2508]	train-mlogloss:2.48747	Test-mlogloss:2.4955
[2509]	train-mlogloss:2.48746	Test-mlogloss:2.49549
[2510]	train-mlogloss:2.48746	Test-mlogloss:2.49549
[2511]	train-mlogloss:2.48745	Test-mlogloss:2.49548
[2512]	train-mlogloss:2.48744	Test-mlogloss:2.49548
[2513]	train-mlogloss:2.48743	Test-mlogloss:2.49548
[2514]	train-mlogloss:2.48742	Test-mlogloss:2.49547
[2515]	train-mlogloss:2.48742	Test-mlogloss:2.49547
[2516]	train-mlogloss:2.48741	Test-mlogloss:2.49547
[2517]	train-mlogloss:2.4874	Test-mlogloss:2.49546
[2518]	train-mlogloss:2.4874	Test-mlogloss:2.49546
[2519]	train-mlogloss:2.48739	Test-mlogloss:2.49546
[2520]	train-mlogloss:2.48738	Test-mlogloss:2.49545
[2521]	train-mlogloss:2.48738	Test-mlogloss:2.49545
[2522]	train-mlogloss:2.48737	Test-mlogloss:2.49544
[2523]	train-mlogloss:2.48736	Test-mlogloss:2.49544
[2524]	train-mlogloss:2.48735	Test-mlogloss:2.49544
[2525]	train-mlogloss:2.48734	Test-mlogloss:2.49543
[2526]	train-mlo

[2666]	train-mlogloss:2.48636	Test-mlogloss:2.49498
[2667]	train-mlogloss:2.48636	Test-mlogloss:2.49498
[2668]	train-mlogloss:2.48635	Test-mlogloss:2.49498
[2669]	train-mlogloss:2.48634	Test-mlogloss:2.49498
[2670]	train-mlogloss:2.48633	Test-mlogloss:2.49497
[2671]	train-mlogloss:2.48633	Test-mlogloss:2.49497
[2672]	train-mlogloss:2.48632	Test-mlogloss:2.49497
[2673]	train-mlogloss:2.48631	Test-mlogloss:2.49497
[2674]	train-mlogloss:2.48631	Test-mlogloss:2.49496
[2675]	train-mlogloss:2.4863	Test-mlogloss:2.49496
[2676]	train-mlogloss:2.4863	Test-mlogloss:2.49496
[2677]	train-mlogloss:2.48629	Test-mlogloss:2.49496
[2678]	train-mlogloss:2.48628	Test-mlogloss:2.49495
[2679]	train-mlogloss:2.48627	Test-mlogloss:2.49495
[2680]	train-mlogloss:2.48627	Test-mlogloss:2.49495
[2681]	train-mlogloss:2.48626	Test-mlogloss:2.49494
[2682]	train-mlogloss:2.48625	Test-mlogloss:2.49494
[2683]	train-mlogloss:2.48625	Test-mlogloss:2.49494
[2684]	train-mlogloss:2.48624	Test-mlogloss:2.49494
[2685]	train-m

[2825]	train-mlogloss:2.4853	Test-mlogloss:2.49454
[2826]	train-mlogloss:2.4853	Test-mlogloss:2.49454
[2827]	train-mlogloss:2.48529	Test-mlogloss:2.49453
[2828]	train-mlogloss:2.48528	Test-mlogloss:2.49453
[2829]	train-mlogloss:2.48528	Test-mlogloss:2.49453
[2830]	train-mlogloss:2.48527	Test-mlogloss:2.49452
[2831]	train-mlogloss:2.48526	Test-mlogloss:2.49452
[2832]	train-mlogloss:2.48526	Test-mlogloss:2.49452
[2833]	train-mlogloss:2.48525	Test-mlogloss:2.49452
[2834]	train-mlogloss:2.48524	Test-mlogloss:2.49451
[2835]	train-mlogloss:2.48524	Test-mlogloss:2.49451
[2836]	train-mlogloss:2.48523	Test-mlogloss:2.49451
[2837]	train-mlogloss:2.48523	Test-mlogloss:2.49451
[2838]	train-mlogloss:2.48522	Test-mlogloss:2.4945
[2839]	train-mlogloss:2.48521	Test-mlogloss:2.4945
[2840]	train-mlogloss:2.48521	Test-mlogloss:2.4945
[2841]	train-mlogloss:2.4852	Test-mlogloss:2.4945
[2842]	train-mlogloss:2.4852	Test-mlogloss:2.49449
[2843]	train-mlogloss:2.48519	Test-mlogloss:2.49449
[2844]	train-mloglos

[2984]	train-mlogloss:2.48431	Test-mlogloss:2.49416
[2985]	train-mlogloss:2.4843	Test-mlogloss:2.49416
[2986]	train-mlogloss:2.48429	Test-mlogloss:2.49415
[2987]	train-mlogloss:2.48429	Test-mlogloss:2.49415
[2988]	train-mlogloss:2.48428	Test-mlogloss:2.49415
[2989]	train-mlogloss:2.48428	Test-mlogloss:2.49415
[2990]	train-mlogloss:2.48427	Test-mlogloss:2.49415
[2991]	train-mlogloss:2.48426	Test-mlogloss:2.49414
[2992]	train-mlogloss:2.48426	Test-mlogloss:2.49414
[2993]	train-mlogloss:2.48425	Test-mlogloss:2.49414
[2994]	train-mlogloss:2.48425	Test-mlogloss:2.49414
[2995]	train-mlogloss:2.48424	Test-mlogloss:2.49413
[2996]	train-mlogloss:2.48423	Test-mlogloss:2.49413
[2997]	train-mlogloss:2.48423	Test-mlogloss:2.49413
[2998]	train-mlogloss:2.48422	Test-mlogloss:2.49413
[2999]	train-mlogloss:2.48421	Test-mlogloss:2.49413
[3000]	train-mlogloss:2.48421	Test-mlogloss:2.49412
[3001]	train-mlogloss:2.4842	Test-mlogloss:2.49412
[3002]	train-mlogloss:2.4842	Test-mlogloss:2.49412
[3003]	train-ml

In [ ]:
print(bst.attributes())

#### 预测验证集

In [ ]:
test_data = datapre.extra_dates(test_data)
test_data['DayOfWeekID'] = DayOfWeekEnc.transform(test_data['DayOfWeek'])
test_data['PdDistrictID'] = PdDistrictEnc.transform(test_data['PdDistrict'])
test_data = datapre.extra_address_for_block(test_data)
test_data = datapre.extra_address_for_suffix(test_data)

In [ ]:
test_data['PositionType'] = PositionTypeEnc.transform(test_data['PdDistrict'])
X_valid = test_data[feature_names]
DX_valid = xgb.DMatrix(data=X_valid)

In [ ]:
y_pred_prob = np.round(bstgpu.predict(DX_valid), 4)

In [ ]:
from sklearn.metrics import  log_loss, accuracy_score

In [ ]:
log_loss(y_test, y_pred_prob, labels=xgbclf.classes_)

In [ ]:
accuracy_score(y_test, y_pred)